# TAREA 4
## Planificación Minera - MI5073
Autor: Jorge Salvo Marín

Profesor: Juan L. Yarmuch

Profesor Auxiliar: Erick Sanhueza

Ayudantes: Gabriel Ojeda y Francisco Arévalo

In [1]:
from pulp import *
import numpy as np
import matplotlib as plt

## Problema de optimización lineal

$$f(x) = \sum_{t \in T} \delta_{t} \sum_{a \in A} w_{a} x_{at} $$
s.t.

1) Precedencia de actividades
$$ x_{ut} \leq \sum_{T \leq t} x_{vt} $$

2) Restricci´on de capacidades
$$ \sum_{a \in A} f_{a}^i  x_{at} \leq M_{t}^i  $$, con $$ t \in T, ~ f^i \in F^{i}, ~ i \in {r,h,c} $$

3) La actividad solo se puede realizar una vez en todo el horizonte temporal.
$$  \sum_{t \in T} x_{at} \leq 1 $$, con $$ a \in A $$

4) Variables
$$ ( x_{a} \in {0, 1} ) , ~ para ~ a \in A-C ~ ~ y ~ ~ ( x_{a} \in [0, 1] ) ~ para ~ a \in C $$



## Definición de problema

In [2]:
agenda = LpProblem("SLS", LpMaximize)

## Parámetros

In [3]:
set_A = list(range(1,24))    #Lista de Actividades (rampa, accesos y caserones)
set_W = { 1:-100 , 2:-100 , 3:-100 , 4:-50 , 5:600 , 6:-50 , 7:200 ,
         8:-110 , 9:-110 , 10:-110 , 11:-110 , 12:-55 , 13:400 , 14:-55 , 15:250,
         16:-120 , 17:-120 , 18:-120 , 19:-120 , 20:-55 , 21:8000 ,
         22:-120 , 23:230 }  #Diccionario de actividades asociadas al costo

T = list(range(1,11))

#Precedencia
set_P = [(2,1),(3,2),(4,3),(5,4),(6,3),(7,6),(8,3),(9,8),(10,9),(11,10),(12,11),(13,12),
        (14,11),(15,14),(16,11),(17,16),(18,17),(19,18),(20,19),(21,20),(22,19),(23,22)]

In [4]:
set_T = list(range(1,11))  #Períodos representados por años en los que se realizan las actividades

set_Fr = {1:300 , 2:300 , 3:300 , 8:300 , 9:300 , 10:300 , 11:300 , 16:300 , 17:200 ,
          18:300, 19:300 }  #Toneladas por seccion rampa
set_Fh = {4:500 , 6:300 , 12:600 , 14:200 , 20:800 , 22:100}  #Toneladas por seccion de avances horizontales (accesos)
set_Fc = {5:1000 , 7:1200 , 13:700 , 15:1600 , 21:2000 , 23:800}  #Toneladas por seccion de caserones

In [5]:
d = 0.1     #Tasa de descuento
M_r = 900   #Capacidad de metros anuales en desarrollos verticales [m/año]
M_h = 900   #Capacidad de metros anuales en desarrollos horizontales [m/año]
M_c = 1500  #Capacidad mina en toneladas por año [t/año]

## Variables

In [6]:
desarrollos = []
for a in set_A:
    if a not in set_Fc:
        desarrollos.append(a)     #Va agregando a la lista el número de la actividad que corresponde a los desarrollos

In [7]:
X = pulp.LpVariable.dicts("x", (desarrollos,set_T), cat="Binary")   #Correspondiente a 0 si el desarrollo no se ha 
                                                                    #explotado en período t y 1 se explotó
Y = pulp.LpVariable.dicts("y", (set_Fc,set_T), lowBound = 0, upBound=1,
                          cat="Continuous")    #Correspondiente a la fracción del caserón que se ha explotado hasta el período t

## Objetivo

In [8]:
agenda += pulp.lpSum( [X[a][t] * set_W[a] / ((1 + d)**t) for a in desarrollos  for t in set_T] +      #Correspondiente a los desarrollos
                     [ Y[c][t] * set_W[c] / ((1 + d)**t) for c in set_Fc  for t in set_T] ),"NPV" #Correspondiente a los Caserones

## Restricciones

### Restricción de precedencia

In [9]:
for t in set_T:
    for (u,v) in set_P:
        if (u in desarrollos) and (v in desarrollos):    #Resticción para los desarrolos (binarios)
            agenda += X[u][t] <= lpSum( X[v][tau] for tau in range(1,t+1) ), f"precedences{u,v,t}" 
        elif (u in desarrollos) and (v in set_Fc):    #Resticción para un desarrollo y un caserón
            agenda += X[u][t] <= lpSum( Y[v][tau] for tau in range(1,t+1) ), f"precedences{u,v,t}" 
        elif (u in set_Fc) and (v in desarrollos):    #Resticción para un caserón y un desarrollo
            agenda += Y[u][t] <= lpSum( X[v][tau] for tau in range(1,t+1) ), f"precedences{u,v,t}" 

### Restricción de Recursos

In [10]:
for t in set_T:
    agenda += lpSum( X[fr][t] * set_Fr[fr] for fr in set_Fr) <= M_r, f"resources_F1_{t}"   #Restricción de capacidad de metros/año rampa
    agenda += lpSum( X[fh][t] * set_Fh[fh] for fh in set_Fh) <= M_h, f"resources_F2_{t}"   #Restricción de capacidad de metros/año accesos
    agenda += lpSum( Y[fc][t] * set_Fc[fc] for fc in set_Fc) <= M_c, f"resources_F3_{t}"   #Restricción de capacidad de toneladas/año mina

### Restricción de Unicidad

In [11]:
for a in desarrollos:
    agenda += lpSum( X[a][t] for t in set_T) <= 1, f"uni__{a}"   #Se debe haber explotado 1 vez en total
    
for fc in set_Fc:
    agenda += lpSum( Y[fc][t] for t in set_T) <= 1, f"completion_{fc}"  #Se debe haber explotado 1 vez en total

## Resolver

In [12]:
agenda.solve()    #Resuelve el problema de optimización lineal

1

## Estado de la Solución

In [13]:
print("Estado de la solución:", LpStatus[agenda.status]) #Muestra el estado de la solución encontrada

Estado de la solución: Optimal


## Valor de las Variables

In [14]:
for i in agenda.variables():
    if i.varValue > 0:
        print(i.name, "=", i.varValue)  #Irá imprimiendo las fracciones de extracción de cada actividad por período

x_10_3 = 1.0
x_11_3 = 1.0
x_12_3 = 1.0
x_14_3 = 1.0
x_16_3 = 1.0
x_17_4 = 1.0
x_18_4 = 1.0
x_19_4 = 1.0
x_1_1 = 1.0
x_20_4 = 1.0
x_22_5 = 1.0
x_2_1 = 1.0
x_3_1 = 1.0
x_4_1 = 1.0
x_6_1 = 1.0
x_8_2 = 1.0
x_9_2 = 1.0
y_13_3 = 1.0
y_15_3 = 0.5
y_15_5 = 0.125
y_15_6 = 0.375
y_21_4 = 0.75
y_21_5 = 0.25
y_23_5 = 1.0
y_5_1 = 1.0
y_7_1 = 0.41666667
y_7_2 = 0.58333333
